In [1]:
!pip install torch torchvision

In [2]:
import torch
print(torch.__version__)


2.1.0+cu121


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
!pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00


# **Load Dataset**

## **Load citeseer**

In [5]:
import torch
from torch_geometric.datasets import Planetoid



citeseer_dataset = Planetoid(root='', name='CiteSeer')

citeseer = citeseer_dataset[0]

Processing...
Done!


## **Load CoraFull**

In [6]:
from torch_geometric.datasets import CoraFull


root = './CoraFull'


CoraFull_dataset = CoraFull(root)

CoraFull_dataset.download()
CoraFull_dataset.process()

CoraFull =  CoraFull_dataset[0]



Processing...
Done!
Using existing file cora.npz


# **Print number of nodes, clases, edges, node features on CoraFull**

In [7]:
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data



# Print statistics for the Cora Full dataset
print("Cora Full Dataset Statistics:")
print("Number of Nodes: ",CoraFull.num_nodes)
print("Number of Edges: ", CoraFull.num_edges)
print("Number of Classes: ", CoraFull_dataset.num_classes)
print("Number of Node Features: ", CoraFull.num_node_features)



Cora Full Dataset Statistics:
Number of Nodes:  19793
Number of Edges:  126842
Number of Classes:  70
Number of Node Features:  8710


# **Print number of nodes, clases, edges, node features on CiteSeer**

In [8]:
# Print statistics for the CiteSeer dataset
print("\nCiteSeer Dataset Statistics:")
print("Number of Nodes: ", citeseer.num_nodes)
print("Number of Edges: ", citeseer.num_edges)
print("Number of Classes: ", citeseer_dataset.num_classes)
print("Number of Node Features: ", citeseer.num_node_features)


CiteSeer Dataset Statistics:
Number of Nodes:  3327
Number of Edges:  9104
Number of Classes:  6
Number of Node Features:  3703


# **Split dataset**

## **split cora full**

In [9]:
from torch_geometric.transforms import RandomNodeSplit

transform = RandomNodeSplit(num_train_per_class=int(CoraFull.num_nodes * 0.7), num_val=int(CoraFull.num_nodes * 0.1), num_test=int(CoraFull.num_nodes * 0.2))
CoraFull = transform(CoraFull)

## **split citeseer**

In [10]:

transform = RandomNodeSplit(num_train_per_class=int(citeseer.num_nodes * 0.7), num_val=int(citeseer.num_nodes * 0.1), num_test=int(citeseer.num_nodes * 0.2))
citeseer = transform(citeseer)

# **important functions**

In [11]:
def train(model, optimizer, criterion, data):
    model.train()
    optimizer.zero_grad()
    data = data.to(device)
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def validate(model, criterion, data):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        out = model(data)
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
    return val_loss.item()

def test(model, criterion, data):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        out = model(data)
        _, pred = torch.max(out, dim=1)
        correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
        acc = correct / data.test_mask.sum().item()
    return acc


# **b)implement multi layer perceptron**

In [12]:
from torch import nn, optim

In [13]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(MultiLayerPerceptron, self).__init__()

        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim


        layers.append(nn.Linear(prev_dim, output_dim))

        self.layers = nn.Sequential(*layers)

    def forward(self, data):
        x =data.x
        return self.layers(x)


## **train and evaluation on cora full dataset**

In [14]:
torch.manual_seed(0)


#archs
architectures = [

    {'hidden_dims': [128, 64, 32 ,16], 'dropout': 0.5},
    {'hidden_dims': [128, 64 ,32], 'dropout': 0.5},
    {'hidden_dims': [128 ,64], 'dropout': 0.5},
    {'hidden_dims': [128], 'dropout': 0.5},

]

best_architecture = None
best_validation_loss = float('inf')


for arch in architectures:

    model = MultiLayerPerceptron(input_dim=CoraFull.num_node_features, hidden_dims=arch['hidden_dims'], output_dim=CoraFull_dataset.num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    # Early stopping parameters
    patience = 5
    min_delta = 0.001
    patience_counter = 0
    best_loss = float('inf')

    # Train the model

    for epoch in range(200):

        loss = train(model, optimizer, criterion, CoraFull)
        val_loss = validate(model, criterion, CoraFull)

        # Check for improvement in validation loss
        if val_loss < best_loss - min_delta:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch} \n')
                break


        print(f'Architecture: {arch}, Epoch: {epoch}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f} ')

    # Evaluate the model
    validation_loss = validate(model= model , criterion= criterion , data = CoraFull)
    print('Architecture: {}, Validation Loss: {:.4f}'.format(arch, validation_loss))

    # Check if this architecture is the best so far
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        best_architecture = arch

print('Best Architecture: {}'.format(best_architecture))

Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 0, Loss: 4.2702, Validation Loss: 4.2483 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 1, Loss: 4.2488, Validation Loss: 4.2066 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 2, Loss: 4.2004, Validation Loss: 4.1345 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 3, Loss: 4.1121, Validation Loss: 4.0677 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 4, Loss: 4.0163, Validation Loss: 3.9701 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 5, Loss: 3.8997, Validation Loss: 3.8652 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 6, Loss: 3.7656, Validation Loss: 3.7515 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 7, Loss: 3.6350, Validation Loss: 3.6662 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 8, Loss

In [15]:
torch.manual_seed(0)

# Train the best model
model = MultiLayerPerceptron(input_dim=CoraFull.num_node_features, hidden_dims=best_architecture['hidden_dims'], output_dim=CoraFull_dataset.num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Early stopping parameters
patience = 5
min_delta = 0.001
patience_counter = 0
best_loss = float('inf')

# Train the model
for epoch in range(200):

        optimizer.zero_grad()

        out = model(CoraFull).to(device)

        loss = criterion(out[CoraFull.train_mask], CoraFull.y[CoraFull.train_mask])

        loss.backward()

        optimizer.step()


        val_loss = criterion(out[CoraFull.val_mask], CoraFull.y[CoraFull.val_mask])

        # Check for improvement in validation loss
        if val_loss < best_loss - min_delta:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break
        print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss))

# Evaluate the best model

test_accuracy = test(model= model , criterion= criterion , data = CoraFull)
print('Best Architecture: {}, Test Accuracy: {:.4f}'.format(best_architecture, test_accuracy))

Epoch: 000, Loss: 4.2478
Epoch: 001, Loss: 3.9125
Epoch: 002, Loss: 3.5630
Epoch: 003, Loss: 3.2016
Epoch: 004, Loss: 2.7984
Epoch: 005, Loss: 2.4078
Epoch: 006, Loss: 2.0475
Epoch: 007, Loss: 1.7208
Epoch: 008, Loss: 1.4273
Epoch: 009, Loss: 1.1760
Epoch: 010, Loss: 0.9686
Epoch: 011, Loss: 0.7971
Epoch: 012, Loss: 0.6585
Epoch: 013, Loss: 0.5447
Epoch: 014, Loss: 0.4491
Epoch: 015, Loss: 0.3695
Epoch: 016, Loss: 0.3047
Epoch: 017, Loss: 0.2524
Epoch: 018, Loss: 0.2100
Epoch: 019, Loss: 0.1761
Epoch: 020, Loss: 0.1494
Early stopping at epoch 21
Best Architecture: {'hidden_dims': [128], 'dropout': 0.5}, Test Accuracy: 0.6309


## **train and evaluation on CiteSeer dataset**

In [16]:
torch.manual_seed(0)

#archs
architectures = [

    {'hidden_dims': [128, 64, 32 ,16], 'dropout': 0.5},
    {'hidden_dims': [128, 64 ,32], 'dropout': 0.5},
    {'hidden_dims': [128 ,64], 'dropout': 0.5},
    {'hidden_dims': [128], 'dropout': 0.5},

]
best_architecture = None
best_validation_loss = float('inf')


for arch in architectures:

    model = MultiLayerPerceptron(input_dim=citeseer.num_node_features, hidden_dims=arch['hidden_dims'], output_dim=citeseer_dataset.num_classes).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Early stopping parameters
    patience = 5
    min_delta = 0.001
    patience_counter = 0
    best_loss = float('inf')

    # Train the model
    for epoch in range(200):

        loss = train(model, optimizer, criterion, citeseer)
        val_loss = validate(model, criterion, citeseer)

        # Check for improvement in validation loss
        if val_loss < best_loss - min_delta:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch} \n')
                break


        print(f'Architecture: {arch}, Epoch: {epoch}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f} ')

    # Evaluate the model
    validation_loss = validate(model= model , criterion= criterion , data = citeseer)
    print('Architecture: {}, Validation Loss: {:.4f} \n'.format(arch, validation_loss))

    # Check if this architecture is the best so far
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        best_architecture = arch

print('Best Architecture: {}'.format(best_architecture))

Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 0, Loss: 1.7976, Validation Loss: 1.7706 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 1, Loss: 1.7733, Validation Loss: 1.7268 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 2, Loss: 1.7128, Validation Loss: 1.6575 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 3, Loss: 1.6056, Validation Loss: 1.5788 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 4, Loss: 1.4670, Validation Loss: 1.4505 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 5, Loss: 1.2915, Validation Loss: 1.3437 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 6, Loss: 1.1335, Validation Loss: 1.3128 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 7, Loss: 1.0078, Validation Loss: 1.3534 
Architecture: {'hidden_dims': [128, 64, 32, 16], 'dropout': 0.5}, Epoch: 8, Loss

In [17]:
torch.manual_seed(0)


# Train the best model
model = MultiLayerPerceptron(input_dim=citeseer.num_node_features, hidden_dims=best_architecture['hidden_dims'], output_dim=citeseer_dataset.num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01) # Reset the optimizer state

# Early stopping parameters

patience = 5
min_delta = 0.001
patience_counter = 0
best_loss = float('inf')

# Train the model
for epoch in range(200):

        loss = train(model, optimizer, criterion, citeseer)
        val_loss = validate(model, criterion, citeseer)

        # Check for improvement in validation loss
        if val_loss < best_loss - min_delta:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break
        print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss))

# Evaluate the best model
test_accuracy = test(model= model , criterion= criterion , data = citeseer)
print('Best Architecture: {}, Test Accuracy: {:.4f}'.format(best_architecture, test_accuracy))

Epoch: 000, Loss: 1.7952
Epoch: 001, Loss: 1.5321
Epoch: 002, Loss: 1.2054
Epoch: 003, Loss: 0.9048
Epoch: 004, Loss: 0.6520
Epoch: 005, Loss: 0.4633
Epoch: 006, Loss: 0.3293
Epoch: 007, Loss: 0.2360
Epoch: 008, Loss: 0.1680
Epoch: 009, Loss: 0.1109
Epoch: 010, Loss: 0.0724
Early stopping at epoch 11
Best Architecture: {'hidden_dims': [128], 'dropout': 0.5}, Test Accuracy: 0.7158
